In [2]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("fruits_nuts", {}, "../Data/fruits_nuts/trainval.json", "../Data/fruits_nuts/images")

In [5]:
from detectron2.data import DatasetCatalog, MetadataCatalog
balloon_metadata = MetadataCatalog.get("fruits_nuts")

In [6]:
dataset_dicts = DatasetCatalog.get("fruits_nuts")

In [8]:
import random 
import cv2
from detectron2.utils.visualizer import Visualizer 

for d in random.sample(dataset_dicts, 3): 
    img = cv2.imread(d["file_name"]) 
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5) 
    vis = visualizer.draw_dataset_dict(d) 
    cv2.imshow("test", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [9]:
from detectron2.engine import DefaultTrainer 
from detectron2.config import get_cfg 
import os 

cfg = get_cfg() 
cfg.merge_from_file("../study/detectron2-master/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" ) 
cfg.DATASETS.TRAIN = ("fruits_nuts",) 
cfg.DATASETS.TEST = () # no metrics implemented for this dataset 
cfg.DATALOADER.NUM_WORKERS = 2 
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 2 
cfg.SOLVER.BASE_LR = 0.02 
cfg.SOLVER.MAX_ITER = ( 300 ) # 300 iterations seems good enough, but you can certainly train longer 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = ( 128 ) # faster, and good enough for this toy dataset 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 # 3 classes (data, fig, hazelnut) 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) 
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) 
trainer.train()

Config '../study/detectron2-master/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
/home/djxc/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/djxc/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/djxc/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (

OrderedDict()

In [10]:
%load_ext tensorboard
%tensorboard --logdir output

In [12]:
from detectron2.engine.defaults import DefaultPredictor
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 加载模型
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("fruits_nuts", )
predictor = DefaultPredictor(cfg)

In [14]:
from detectron2.utils.visualizer import ColorMode 

for d in random.sample(dataset_dicts, 3):   
    im = cv2.imread(d["file_name"]) 
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1], metadata=balloon_metadata,
                   scale=0.8,  instance_mode=ColorMode.IMAGE_BW)# remove the colors of unsegmented pixels  ) 
    v = v.draw_instance_predictions(outputs["instances"].to("cpu")) 
    cv2.imshow("test", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows() 